In [30]:
# Library imports
import json
import numpy as np
import sys

## **Open Data**

Data and Solution files. Testing the first instance. TODO: expand to other JSON files.

In [11]:
with open('data/W1-01.json') as f:
    data = json.load(f)

with open('solutions/sol-W1-01.json') as f:
    solution = json.load(f)

Some hyperparameters to adjust:

In [12]:
OVERTIME_P = 1.0  # willingness of nurses to work overtime
ABSENCE_P = 0.5   # probability of absence

In [37]:
# Helpful constants
NUM_DAYS = len(solution["Solution"][0]["Assignments"]) # For W1-01, should be 30 days
NUM_NURSES = len(solution["Solution"])
OFF_SHIFTS = ["R", "A", "S"]

## **Parsing Nurse Data**

From the Data and Solution JSONs, we will parse the following data:
1. `Nurse` class consists of:
    - ID: i.e. `nurse_1`
    - Assignments: list of shifts
    - Contract ID
    - Overtime shifts
    - Overtime willingness
    - Contract details
    - Max assignments
    - Remaining available days
2. `nurses_dict`: all personal info for each nurse
3. `ALL_NURSEIDS_SET`: set of all nurse IDs
4. `nurse_status_dict`: dictionary of set of nurses by status
    - `working_nurses`: set of working nurses for each day (stored in array, index = day)
    - `standby_nurses`: set of nurses on standby
    - `resting_nurses`: set of resting nurses with remaining shifts
    - `overtime_nurses`: set of resting nurses with no remaining shifts (will require overtime)
5. `init_state`: initial num_days x num_nurses matrix based on solution
    - 0: Absent
    - 1: Working (has a shift)
    - 2: Standby ("S")
    - 3: Resting and has remaining overtime shifts ("R" and `remaining_available_days > 0`)
    - 4: Overtime: resting and has no remaining overtime shifts ("R" and `remaining_available_days == 0`)

### **`Nurse` Class**

In [20]:
class Nurse():
    def __init__(self, id, index, assignments, contract_id, shiftoffrequests):
        self.id = id      # ex: Nurse_1
        self.index = index       # ex: 0 (associated with place in JSON lists)
        self.assignments = assignments # list of shifts
        
        self.overtime_shifts = 0 # nobody has overtime for now
        self.overtime_willingness = 1 # always accept overtime TODO: Change this to use hyperparameter
        
        # contract details
        self.contract_id = contract_id # either Contract_1 or Contract_2
        self.contract_details = self.parse_contract()
        self.max_assignments = self.contract_details["MaximumNumberOfAssignments"]
        
        # assignments
        self.num_assignments = len([x for x in self.assignments if x not in OFF_SHIFTS])
        self.remaining_available_days = self.max_assignments - self.num_assignments
        
        # shiftoffrequests
        self.shiftoffrequests = shiftoffrequests
    
    def parse_contract(self):
        contract_details = {}
        if self.contract_id == "Contract_1":
            contract_details = data["Contracts"][0]
        elif self.contract_id == "Contract_2":
            contract_details = data["Contracts"][1]
        else:
            print("Invalid contract id.")
            sys.exit(1)
        return contract_details
    
    def sample_overtime(self):
        # TODO: CHANGE TO DISTRIBUTION LATER
        return self.overtime_willingness
    
    def list_assignments(self):
        return self.assignments
    
    def list_shiftoffrequests(self):
        return self.shiftoffrequests

TODO: IGNORE FOR NOW

In [14]:
# TODO: potentially use, but ignore for now
# collect all shiftoffrequests as dictionary: {Nurse_1: 5,Night}
def get_shiftoffrequests(data):
    shiftoffrequests = dict()
    for request in data["Shiftoffrequests"]:
        if request["id"] not in shiftoffrequests:
            shiftoffrequests[request["id"]] = list()
        shiftoffrequests[request["id"]].append({request["day"]:request["shift"]})
    return shiftoffrequests

### **Nurse Dictionary and ID Set**

In [22]:
def get_nurse_dict(solution, shiftoffrequests):
    ALL_NURSEIDS_SET = set()
    nurses_dict = dict()
        
    # make nurse objects, stores shifts in list
    for index, nurse_sol in enumerate(solution["Solution"]):  
        nurse_id = nurse_sol["id"]
        assignments = [day["shift"] for day in nurse_sol["Assignments"]]
        contract_id = data["Nurses"][index]["contract_id"]
        if nurse_id in shiftoffrequests:
            nurse_requests = shiftoffrequests[nurse_id]
        else: nurse_requests = []
        
        # Add to nurse dictionary
        new_nurse = Nurse(nurse_id, index, assignments,contract_id, nurse_requests)
        nurses_dict[nurse_id] = new_nurse
        
        # Add to nurse ID set
        ALL_NURSEIDS_SET.add(nurse_id) 
    
    return ALL_NURSEIDS_SET, nurses_dict

### **Nurse Status Dictionary**

In [35]:
def get_nurse_status_dict(nurses_dict):
    # set nurse ID of working nurses per day
    working_nurses = [set([id for id, nurse in nurses_dict.items() if nurse.assignments[day] not in OFF_SHIFTS]) 
                    for day in range(NUM_DAYS)]
    absent_nurses = [set([id for id, nurse in nurses_dict.items() if nurse.assignments[day] == "A"]) 
                    for day in range(NUM_DAYS)]
    standby_nurses = [set([id for id, nurse in nurses_dict.items() if nurse.assignments[day] == "S"]) 
                    for day in range(NUM_DAYS)]
    resting_nurses = [set([id for id, nurse in nurses_dict.items() if nurse.assignments[day] == "R" and nurse.remaining_available_days > 0]) 
                    for day in range(NUM_DAYS)]
    overtime_nurses = [set([id for id, nurse in nurses_dict.items() if nurse.assignments[day] == "R" and nurse.remaining_available_days == 0]) 
                    for day in range(NUM_DAYS)]

    # {nurse status: list of nurse ID sets (index = day)}
    nurse_status_dict = dict()
    nurse_status_dict["working"] = working_nurses
    nurse_status_dict["absent"] = absent_nurses
    nurse_status_dict["standby"] = standby_nurses
    nurse_status_dict["resting"] = resting_nurses
    nurse_status_dict["overtime"] = overtime_nurses

    return nurse_status_dict

In [36]:
# TODO: may revisit this logic
# # list of sets of available nurses per day
# available_nurses = []
# for day in range(NUM_DAYS):
#     print("day ",day)
#     staffed_set = working_nurses[day]
#     unstaffed_set = ALL_NURSEIDS_SET.difference(staffed_set)
#     # print(unstaffed_set)
    
#     # check unstaffed nurses for how many total assignments
#     available_set = set()
#     for unstaffed_nurse in unstaffed_set:
#         # print(nurses_dict[unstaffed_nurse].id)
#         # print(nurses_dict[unstaffed_nurse].assignments)
#         # print(nurses_dict[unstaffed_nurse].contract_id)
#         # print(nurses_dict[unstaffed_nurse].max_assignments)
#         # print(nurses_dict[unstaffed_nurse].num_assignments)
#         # print(nurses_dict[unstaffed_nurse].remaining_available_days)
#         # print(nurses_dict[unstaffed_nurse].shiftoffrequests)
        
#         if (nurses_dict[unstaffed_nurse].remaining_available_days > 0 
#             and str(day+1) not in nurses_dict[unstaffed_nurse].shiftoffrequests
#            and nurses_dict[unstaffed_nurse].assignments[day] == "S"):
#                 available_set.add(unstaffed_nurse)
    
#     # WHAT DOES "R" MEAN. CAN IT BE BREACHED. HOW TO TELL IF NIGHT SHIFT/VIOLATION.
#     # ONLY INCLUDE "S" (standby??) in available set??
#     print(available_set)
#     available_nurses.append(available_set)
    
# print([len(day) for day in available_nurses])

### **Initial State**

Initial num_days x num_nurses matrix based on solution:
- 0: Absent
- 1: Working (has a shift)
- 2: Standby ("S")
- 3: Resting (resting and has remaining overtime shifts ("R" and `remaining_available_days > 0`))
- 4: Overtime (resting and has no remaining overtime shifts ("R" and `remaining_available_days == 0`))

In [39]:
def get_init_state(nurse_status_dict, nurses_dict):
    init_state = np.zeros((NUM_DAYS, NUM_NURSES))
    for status, days in nurse_status_dict.items():
        for day, nurses in enumerate(days):
            for nurse_id in nurses:
                nurse = nurses_dict[nurse_id]
                if status == "absent":
                    init_state[day][nurse.index] = 0
                if status == "working":
                    init_state[day][nurse.index] = 1
                if status == "standby":
                    init_state[day][nurse.index] = 2
                if status == "resting":
                    init_state[day][nurse.index] = 3
                if status == "overtime":
                    init_state[day][nurse.index] = 4

    return init_state


## **Simulation Environment**

Putting it all together!

In [13]:
class SchedulingEnv():
    def __init__(self, data_path, solution_path, OVERTIME_P = OVERTIME_P, ABSENCE_P = ABSENCE_P):
        # Open JSONs
        with open(data_path) as f:
            self.data = json.load(f)
        with open(solution_path) as f:
            self.solution = json.load(f)

        # Parse data and solution
        self.init_shiftoffrequests()
        self.init_nurse_dict()
        self.init_nurse_status_dict()
        self.init_init_state()
    
    def init_shiftoffrequests(self):
        self.shiftoffrequests = dict()
        for request in self.data["Shiftoffrequests"]:
            if request["id"] not in self.shiftoffrequests:
                self.shiftoffrequests[request["id"]] = list()
            self.shiftoffrequests[request["id"]].append({request["day"]:request["shift"]})
    
    def init_nurse_dict(self):
        self.ALL_NURSEIDS_SET = set()
        self.nurses_dict = dict()
            
        # make nurse objects, stores shifts in list
        for index, nurse_sol in enumerate(self.solution["Solution"]):  
            nurse_id = nurse_sol["id"]
            assignments = [day["shift"] for day in nurse_sol["Assignments"]]
            contract_id = data["Nurses"][index]["contract_id"]
            if nurse_id in self.shiftoffrequests:
                nurse_requests = self.shiftoffrequests[nurse_id]
            else: nurse_requests = []
            
            # Add to nurse dictionary
            new_nurse = Nurse(nurse_id, index, assignments,contract_id, nurse_requests)
            self.nurses_dict[nurse_id] = new_nurse
            
            # Add to nurse ID set
            self.ALL_NURSEIDS_SET.add(nurse_id) 

    def init_nurse_status_dict(self):
        # set nurse ID of working nurses per day
        working_nurses = [set([id for id, nurse in self.nurses_dict.items() if nurse.assignments[day] not in OFF_SHIFTS]) 
                        for day in range(NUM_DAYS)]
        absent_nurses = [set([id for id, nurse in self.nurses_dict.items() if nurse.assignments[day] == "A"]) 
                        for day in range(NUM_DAYS)]
        standby_nurses = [set([id for id, nurse in self.nurses_dict.items() if nurse.assignments[day] == "S"]) 
                        for day in range(NUM_DAYS)]
        resting_nurses = [set([id for id, nurse in self.nurses_dict.items() if nurse.assignments[day] == "R" and nurse.remaining_available_days > 0]) 
                        for day in range(NUM_DAYS)]
        overtime_nurses = [set([id for id, nurse in self.nurses_dict.items() if nurse.assignments[day] == "R" and nurse.remaining_available_days == 0]) 
                        for day in range(NUM_DAYS)]

        # {nurse status: list of nurse ID sets (index = day)}
        self.nurse_status_dict = dict()
        self.nurse_status_dict["working"] = working_nurses
        self.nurse_status_dict["absent"] = absent_nurses
        self.nurse_status_dict["standby"] = standby_nurses
        self.nurse_status_dict["resting"] = resting_nurses
        self.nurse_status_dict["overtime"] = overtime_nurses
    
    def init_init_state(self):
        self.init_state = np.zeros((NUM_DAYS, NUM_NURSES))
        for status, days in self.nurse_status_dict.items():
            for day, nurses in enumerate(days):
                for nurse_id in nurses:
                    nurse = self.nurses_dict[nurse_id]
                    if status == "absent":
                        self.init_state[day][nurse.index] = 0
                    if status == "working":
                        self.init_state[day][nurse.index] = 1
                    if status == "standby":
                        self.init_state[day][nurse.index] = 2
                    if status == "resting":
                        self.init_state[day][nurse.index] = 3
                    if status == "overtime":
                        self.init_state[day][nurse.index] = 4

In [96]:
# # run sim
# # sample an absence on one day
# # reward function 

# class SchedulingEnv():
#     def __init__(self, data_path, solution_path):
#         # load instance
#         with open(data_path) as f:
#             self.data = json.load(f)

#         # load given solutions
#         with open(solution_path) as f:
#             self.solution = json.load(f)
        
#         # process all shift off requests; make dictionary indexed by nurse id
#         self.shiftoffrequests = dict()
#         self.parse_shiftoffrequests()
        
#         # instantiate + populate dict of all nurse objects; also store set of all nurse ids
#         self.ALL_NURSEIDS_SET = set()
#         self.nurses_dict = dict()
#         self.parse_nurse_objects()
        
#         # metadata
#         self.num_nurses = len(nurses_dict)
#         self.num_days = self.data["numberOfDays"]
        
#         # track current schedule
#         self.original_schedule 
#         self.current_schedule
        
        
#     def parse_shiftoffrequests(self):
#         for request in self.data["Shiftoffrequests"]:
#             if request["id"] not in self.shiftoffrequests:
#                 self.shiftoffrequests[request["id"]] = list()
#             self.shiftoffrequests[request["id"]].append({request["day"]:request["shift"]})
            
#     def parse_nurse_objects(self):
#         index = 0; # sometimes the json doesn't have key, just based on index
#         for nurse_sol in self.solution["Solution"]:    
#             nurse_id = nurse_sol["id"]
#             assignments = [day["shift"] for day in nurse_sol["Assignments"]]
#             contract_id = self.data["Nurses"][day]["contract_id"]
#             if nurse_id in self.shiftoffrequests:
#                 nurse_requests = self.shiftoffrequests[nurse_id]
#             else: nurse_requests = []

#             new_nurse = Nurse(nurse_id, index, assignments,contract_id, nurse_requests)
#             self.nurses_dict[nurse_id] = new_nurse # populate dict

#             ALL_NURSEIDS_SET.add(nurse_id) # populate set
            
#             index+=1
